# 1. Import library

In [54]:
import requests
# !pip install haversine
import haversine
import datetime
from dateutil.parser import parse
import pandas as pd
from ast import literal_eval
import time

# 2. importing Classes

### 2-1. Date Reformat Class

In [55]:
class Find:
    def f_date(self,p_date):
        from dateutil.parser import parse
        p_date= parse(p_date)
        p_date = p_date.strftime('%Y-%m-%d-%a')
        return p_date

### 2-2. Cal_Distance Class

In [56]:
class Cal_Distance:
    def getLatLng(self,p_start,p_dn_name):
        import requests
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_s_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_s_Lat = float(p_s_places[0]['y'])
        p_s_Lng = float(p_s_places[0]['x'])
        p_s_LatLng=(p_s_Lat,p_s_Lng)

        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_dn_name)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_d_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_d_Lat = float(p_d_places[0]['y'])
        p_d_Lng = float(p_d_places[0]['x'])
        p_d_LatLng=(p_d_Lat,p_d_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
        return r_s_d

    


# select_number = int(input('몇 번째 주소인지 입력하세요 : '))

# print(places[select_number])
    

### 2-3 Evaluation Class

In [57]:
#  기준값적용 (평가기준값)
#  - 기준값 지정 (총 5점)
#  - 평가값을 받는 것 = 이동수단 / 비용 / 거리 ** 서울역 기준 /   

class Evaluation:
    def evalue_costs(self,p_cost,p_length_of_stay): #관광데이터랩 1회 평균 국내여행지출 금액 133,000원 정규분포도 활용
        if p_cost/p_length_of_stay <= 50000: #50000
            return 5
        elif 50000 < p_cost/p_length_of_stay <= 100000: #50001~100000
            return 4
        elif 100000 < p_cost/p_length_of_stay <= 150000: #100001~150000
            return 3
        elif 150000 < p_cost/p_length_of_stay <= 200000: #150001~200000
            return 2
        else: #200001이상.
            return 1
        
    def evalue_by_distance(self,p_transportation,r_s_d): #이동수단을 직선거리로 평가.
        if p_transportation == "도보":
            if r_s_d <= 1:
                return 5
            elif 1 < r_s_d <= 2:
                return 4
            elif 2 < r_s_d <= 3:
                return 3
            elif 3 < r_s_d <= 4:
                return 2
            else:
                return 1
            
        elif p_transportation == "지역버스" or p_transportation == "지하철" :
            if r_s_d <= 10:
                return 5
            elif 10 < r_s_d <= 20:
                return 4
            elif 20 < r_s_d <= 30:
                return 3
            elif 30 < r_s_d <= 40:
                return 2
            else:
                return 1
            
        elif p_transportation == "자차" or p_transportation == "자동차":
            if r_s_d <= 50:
                return 5
            elif 50 < r_s_d <= 100:
                return 4
            elif 100 < r_s_d <= 150:
                return 3
            elif 150 < r_s_d <= 200:
                return 2
            else:
                return 1
            
        elif p_transportation == "고속버스":
            if r_s_d <= 70:
                return 5
            elif r_s_d <= 140:
                return 4
            elif r_s_d <= 210:
                return 3
            elif r_s_d <= 280:
                return 2
            else:
                return 1
            
        elif p_transportation == "전철" or p_transportation == "KTX":
            if r_s_d <= 90:
                return 5
            elif 90 < r_s_d <= 180:
                return 4
            elif 180 < r_s_d <= 270:
                return 3
            elif 270 < r_s_d <= 360:
                return 2
            else:
                return 1
            
        elif p_transportation == "비행기":
            if r_s_d <= 120:
                return 1
            elif 120 < r_s_d <= 200:
                return 2
            elif 200 < r_s_d <= 280:
                return 3
            elif 280< r_s_d <= 360:
                return 4
            else:
                return 5
        
        elif p_transportation == "비행기" and p_start == "제주도":
            return 5
        
    def evalue_by_satisfaction(self,p_satisfaction): #관광만족도 개인 평가 
        if p_satisfaction <= 5:
            return p_satisfaction
        else:
            return 5
    
            
        

2-4-1 Method(Evaluation,Cal_Distance,Find) class 상속

In [58]:
class Method(Evaluation,Cal_Distance,Find):
    pass

### 2-4 Data Record Class

In [59]:
class Data_Record:
    def first_record(self):
        with open("data_repository.txt","w") as data:   
            print("첫번째 기록입니다.")
            while True:
                #Method 클래스 지정. 
                m = Method()
                #입력 데이터값 
                print('\n')
                p_date = input("여행날짜 ex)20210701 : ")
                f_p_date = m.f_date(p_date)
                
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/등 : ")
                p_start = input("출발 장소를 입력하세요 : ")
                p_dn_name = input("여행목적지(관광지이름) : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요. 도보/지역버스(지하철)/자동차/고속버스/전철(KTX)/비행기 : ")
                p_length_of_stay = int(input("총 여행 기간 : __ 일 : "))
                p_cost = int(input("총 지출 금액 : __ 원 : "))
                p_satisfaction = float(input("관광만족도는 어떠셧습니까? <매우좋았음:5 ~ 비추천:1> : "))
                print('\n\n')
                input_data = {'여행날짜':f_p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지이름':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction}      
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                input_value = {'지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                
                #데이터 기록 값
                past_data = {'여행날짜':p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지이름':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction,
                             '지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                print("기록하신 내용입니다.")
                print(input_data)
                print('\n')
                print("기록한 내용에 대한 평가입니다.")
                print(input_value)
                
                correct = input("이 기록이 맞으시면 '예', 아니면 '아니오' 을 눌러주세요 : ")
                correct
                if correct == "예":
                    print('\n')
                    print("기록하신 내용이 저장됩니다.")
                    past_data = str(past_data)
                    data.write(past_data +'\n')
                else:
                    print('\n')
                    print("기록하신 내용이 삭제됩니다.")
                
                Question_end = input("여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                elif Question_end == "아니오":
                    while True:
                        End_Q = input("종료하시겠습니까? 예/아니오 : ")
                        if End_Q == "예":
                            print("기록이 종료되었습니다.")
                            print('\n')
                            print("추가 기록된 내용입니다.")
                            print(past_data)
                            break
                        elif End_Q == "아니오":
                            Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                            break
                        else:
                            print("잘못 입력하셨습니다.")
                            continue
                            
        with open("data_repository.txt",'r') as data_r: data_r = data_r.read()
        return print(data_r,sep='\n')
                    
                        
                    
    def another_record(self):
        with open("data_repository.txt","a") as data:    
            print("기존 데이터에 기록합니다.")
            while True:
                #Method 클래스 지정
                m = Method()
                #입력 데이터값 
                print('\n')
                p_date = input("여행날짜 ex)20210701 : ")
                f_p_date = m.f_date(p_date)
                print(f_p_date)
                p_type = input("관광 유형을 고르세요. 도시/바다/캠핑/등 : ")
                p_start = input("출발 장소를 입력하세요 : ")
                p_dn_name = input("여행목적지(관광지이름) : ")
                
                #r_s_d 값
                r_s_d = m.getLatLng(p_start,p_dn_name)
                
                p_transportation = input("이동수단을 고르세요.(택1) 도보/지역버스(지하철)/자동차/고속버스/전철(KTX)/비행기 : ")
                p_length_of_stay = int(input("총 여행 기간 : __ 일 : "))
                p_cost = int(input("총 지출 금액 : __ 원 : "))
                p_satisfaction = float(input("관광만족도는 어떠셧습니까? <매우좋았음:5 ~ 비추천:1> : "))
                print('\n\n')
                input_data = {'여행날짜':f_p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지이름':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction}
                
                
                #평가 데이터 기록값
                ev_c = m.evalue_costs(p_cost,p_length_of_stay)
                ev_d = m.evalue_by_distance(p_transportation,r_s_d)
                ev_s = m.evalue_by_satisfaction(p_satisfaction)
                
                
                #데이터 기록 값
                past_data = {'여행날짜':p_date,
                             '관광유형':p_type,
                             '출발지':p_start,
                             '관광지이름':p_dn_name,
                             '이동수단':p_transportation,
                             '관광기간':p_length_of_stay,
                             '총비용':p_cost,
                             '만족도':p_satisfaction,
                             '지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
                
                input_value = {'지출비용평가점수':ev_c,
                             '이동비율평가점수':ev_d,
                             '만족도평가점수':ev_s}
            
                print("기록하신 내용입니다.")
                print(input_data)
                print('\n')
                
                print("기록한 내용에 대한 평가입니다.")
                print(input_value)
                
                correct = input("이 기록이 맞으시면 '예', 아니면 '아니오' 을 눌러주세요 : ")
                correct
                if correct == "예":
                    print('\n')
                    print("기록하신 내용이 저장됩니다.")
                    print('\n')
                    past_data = str(past_data)
                    data.write(past_data +'\n')
                else:
                    print('\n')
                    print("기록하신 내용이 삭제됩니다.")
                    
                Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                if Question_end == '예':
                    continue
                elif Question_end == "아니오":
                    while True:
                        End_Q = input("종료하시겠습니까? 예/아니오 : ")
                        if End_Q == "예":
                            print("기록이 종료되었습니다.")
                            print('\n')
                            print("추가 기록된 내용입니다.")
                            print(past_data)
                            break
                        elif End_Q == "아니오":
                            Question_end = input("다른 여행지를 기록하시겠습니까? 예/아니오 : ")
                            break
                        else:
                            print("잘못 입력하셨습니다.")
                            continue
                break


# Method Class

##### CAL_DISTANCE (getLatLng) Class

class Cal_Distance:
    def getLatLng(self,p_start,p_dn_name):
        import requests
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_start)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_s_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_s_Lat = float(p_s_places[0]['y'])
        p_s_Lng = float(p_s_places[0]['x'])
        p_s_LatLng=(p_s_Lat,p_s_Lng)

        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(p_dn_name)
        headers = {"Authorization": "KakaoAK ac29d0b5ff83ba31f85e0d6f9f38e797"}
        p_d_places = requests.get(url, headers = headers).json()['documents']
        
        # print(p_s_place) # 리스트내 사전이 여러개가 있을때, 줄바꿈해서 출력방법 모름 --> 알면 UI 상승할 듯
        # select_number = int(input('몇 번째 주소인지 입력하세요 : '))
        # float(p_s_places[select_number]) ~
        
        p_d_Lat = float(p_d_places[0]['y'])
        p_d_Lng = float(p_d_places[0]['x'])
        p_d_LatLng=(p_d_Lat,p_d_Lng)

        #위경도 차이 구하기 -- float값에서 직선거리 int로 변환 후 
        #상대적 직선거리 (relative straight distance 변수에 지정)
        from haversine import haversine
        r_s_d = int(haversine(p_s_LatLng,p_d_LatLng))
        return r_s_d

    


 select_number = int(input('몇 번째 주소인지 입력하세요 : '))

 print(places[select_number])
    

# Program Class

In [40]:
# a = Record()

# Question = input("처음 기록합니까?  예/아니오 : ")
# if Question == "예":
#     a.first_record()
# else:
#     Another_Question = input("기존 데이터에 기록하겠습니까? 예/아니오 : ")
#     if Another_Question == "예":
#         a.another_record()

In [60]:
class Record_Program:
    def start_program(self):
        r = Data_Record()
        print("기록 프로그램이 실행됩니다.")
        print('\n')
        print("-----------------------------------------------------------------------")
        print("""기록 프로그램은 사용자의 여행기록을 기록하고 평가합니다.\n평가기준은 다음과 같습니다.""")
        print("""평가기준 1 : 출발지부터 목적지까지의 직선거리에 대한 이동수단별 평가\n평가기준 2 : 여행 1일 동안 사용한 비용에 대한 평가\n평가기준 3 : 여행만족도에 대한 주관적 평가""")
        print("""평가된 데이터는 관광기록 순위 혹은 탐색에 이용됩니다.\n많은 이용 부탁드립니다. -김광훈""")
        print("-----------------------------------------------------------------------")
        time.sleep(0.5)
        Question = input("처음 기록합니까?  예/아니오 : ")
        if Question == "예":
            print('\n')
            print("================<중요>================")
            print("처음 기록하신다면 기존 기록은 사라집니다.")
            print("======================================")
            om_Question = input("""기존 데이터 삭제 후, 처음부터 기록하시려면 <기록>를 작성해주세요.\n원치 않으시면 <스페이스 한 번> 입력해주세요 : """)
            if om_Question == "기록":
                r.first_record()
            elif om_Question == " ":
                while True:
                    om2_Question = input("""처음으로 돌아가시려면 <처음>을, 기존 데이터 삭제 후, 처음부터 기록하시려면 <기록>을 작성해주세요.\n종료를 원하시면 <종료>를 입력하세요. : """)
                    if om2_Question == "기록":
                        print('\n')
                        r.first_record()
                        break
                    elif om2_Question == "처음":
                        print('\n\n')
                        Record_Program.start_program(self)
                        break
                    elif om2_Question == "종료":
                        print("-----------------------------------------------------------------------")
                        print('기록 프로그램을 종료합니다')
                        break
                    else:
                        continue
                
        else:
            time.sleep(0.2)
            Another_Question = input("기존 데이터에 기록하겠습니까? 예/아니오 : ")
    
            if Another_Question == "예":
                r.another_record()
                
            elif Another_Question == "아니오":
                print("<아니오>를 입력하셨습니다.")
                while True:
                    oma_Question = input("처음으로 돌아가시려면 <처음>을, 종료하시려면 <종료>를 입력하세요. : ")
                    if oma_Question == "처음":
                        Record_Program.start_program(self)
                        break
                    elif oma_Question == '종료':
                        print("-----------------------------------------------------------------------")
                        print('기록 프로그램을 종료합니다')
                        break
                    else:
                        print("잘못 입력하셨습니다.")
                        continue   
                        
            else:
                print("-----------------------------------------------------------------------")
                print('기록 프로그램을 종료합니다')

In [42]:
t = Record_Program()
t.start_program()

기록 프로그램이 실행됩니다.


-----------------------------------------------------------------------
기록 프로그램은 사용자의 여행기록을 기록하고 평가합니다.
평가기준은 다음과 같습니다.
평가기준 1 : 출발지부터 목적지까지의 직선거리에 대한 이동수단별 평가
평가기준 2 : 여행 1일 동안 사용한 비용에 대한 평가
평가기준 3 : 여행만족도에 대한 주관적 평가
평가된 데이터는 관광기록 순위 혹은 탐색에 이용됩니다.
많은 이용 부탁드립니다. -김광훈
-----------------------------------------------------------------------
처음 기록합니까?  예/아니오 : 아니오
기존 데이터에 기록하겠습니까? 예/아니오 : 아니오
<아니오>를 입력하셨습니다.
처음으로 돌아가시려면 <처음>을, 종료하시려면 <종료>를 입력하세요. : 종료
-----------------------------------------------------------------------
기록 프로그램을 종료합니다


# Top 10 Ranking Program Class

###### If program begins, top 10 list will showed. and choose one of the programs which Record and Searching

In [61]:
class Loading:
    def load_data(self):
        with open('data_repository.txt') as f:
            data = f.readlines()
            data = [literal_eval(x.strip()) for x in data]   
            data = pd.DataFrame(data)
            data['평가점수총합'] = data['지출비용평가점수'] + data['이동비율평가점수'] + data['만족도평가점수']
            data['평균점수'] = data['평가점수총합'] / 3
            data = data.round({'평균점수':2})
            data['평가순위'] = data['평균점수'].rank(ascending=False,method='dense')
            data['평가순위'] = data['평가순위'].astype(int)
            data = data.sort_values(by='평가순위')
            data = data.set_index('평가순위',drop=True,append=False)
            data = data.to_csv("data")
            data = pd.read_csv("data")
            data.reset_index(drop=True)
            return 
    
    def top10(self):
        data = pd.read_csv("data")
        data.reset_index(drop=True)
        top_10 = data[['평가순위','관광유형','관광지이름','평균점수']]
        top_10.to_csv("top10")
        top_list = pd.read_csv("top10",index_col=0)
        top_list = top_list.set_index('평가순위')
        top_list.to_csv("top10")
        top = pd.read_csv('top10',index_col=0)
        return top.head(10)


# Top10 function

In [62]:
L = Loading()
L.load_data()
L.top10()

,관광유형,관광지이름,평균점수
평가순위,,,
1,술,대부도,4.33
2,스키,홍천비발디파크,4.27
3,바다,동해시종합버스터미널,4.00
3,도시,부산,4.00
3,도시,제주도,4.00
4,휴가,지리산,3.90
5,산,인왕산,3.80
6,도시,전주한옥마을,3.77
7,바다,제주도,3.67


# Searching Program Class

In [44]:
# Question_index = input("유형은 무엇입니까? ")
# # Question_Name = input("이름은 무엇입니까? ")
# index = data['관광유형'] == Question_index
# # Name = data["관광지이름"] == Question_Name
# Find_S = data[index]
# # Find_S = data[index&Name]
# Find_S = Find_S.reset_index()
# Find_S = Find_S.drop(['평가순위','여행날짜','출발지','이동수단','관광기간','총비용','만족도','지출비용평가점수',
#                       '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
# Answer = Find_S.groupby(["관광유형","관광지이름"]).mean()
# Answer.to_csv("Answer")
# Answer = pd.read_csv("Answer")
# Answer

In [45]:
L = Loading()

In [53]:
class Searching_data:
    def searching_data(self):
        print("데이터를 불러오는 중입니다.")
        while True:
            print("탐색을 시작합니다.")
            L.load_data()
            time.sleep(1)
            print("\n")
            print("========================================<목록>========================================")
            print("1) 여행날짜 2) 관광유형 3) 출발지 4) 관광지이름 5) 이동수단 6) 관광기간 7) 총비용 8) 만족도")
            print("======================================================================================")
            s_Question = input("탐색하고 싶은 목록값을 입력하세요 :  ")
            if s_Question == "여행날짜":
                s_date = input()
                s_data = data['여행날짜'] == 

SyntaxError: unexpected EOF while parsing (<ipython-input-53-387090eb5564>, line 13)

In [50]:
#Dummy
D = Searching_data()
D.searching_data()

탐색을 시작합니다.
데이터를 불러오는 중입니다.


========================================<목록>========================================
1) 여행날짜 2) 관광유형 3) 출발지 4) 관광지이름 5) 이동수단 6) 관광기간 7) 총비용 8) 만족도
탐색하고 싶은 목록값을 입력하세요 :  여행날짜


NameError: name 'data' is not defined

In [22]:
Question_index = input("유형은 무엇입니까? ")
Question_Name = input("이름은 무엇입니까? ")
index = data['관광유형'] == Question_index
Name = data["관광지이름"] == Question_Name
Find_S = data[index&Name]
Find_S = Find_S.reset_index()
Find_S = Find_S.drop(['평가순위','여행날짜','출발지','이동수단','관광기간','총비용','만족도','지출비용평가점수',
                      '이동비율평가점수','만족도평가점수','평가점수총합','index'],axis=1)
Answer = Find_S.groupby(["관광유형","관광지이름"]).mean()
Answer.to_csv("Answer")
Answer = pd.read_csv("Answer")
Answer

유형은 무엇입니까? 도시
이름은 무엇입니까? 제주도


,관광유형,관광지이름,평균점수
0,도시,제주도,3.835
